In [2]:
import pandas as pd
df = pd.read_csv('../fake_job_postings.csv')


In [3]:
binary_cols = [col for col in df.columns if df[col].nunique() == 2]
df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


In [4]:
# Check for null values
grouped_null_percentage = df.groupby('fraudulent').apply(lambda x: x.isnull().mean() * 100)
grouped_null_percentage = grouped_null_percentage.T.round(2)
print(grouped_null_percentage)


fraudulent               0      1
job_id                0.00   0.00
title                 0.00   0.00
location              1.92   2.19
department           64.75  61.32
salary_range         84.45  74.25
company_profile      15.99  67.78
description           0.00   0.12
requirements         14.94  17.78
benefits             40.25  42.03
telecommuting         0.00   0.00
has_company_logo      0.00   0.00
has_questions         0.00   0.00
employment_type      18.98  27.83
required_experience  38.88  50.23
required_education   44.99  52.08
industry             27.20  31.76
function             35.96  38.91
fraudulent            0.00   0.00


In [5]:
# Deduplicate based on all columns except 'job_id'
df_dedup = df.drop_duplicates(subset=[col for col in df.columns if col != 'job_id'])
print(f"Original shape: {df.shape}, Deduplicated shape: {df_dedup.shape}")

Original shape: (17880, 18), Deduplicated shape: (17599, 18)


In [6]:
# Columns to consider for identifying duplicates (ignore 'job_id' and 'fraudulent')
cols_to_check = df.columns.difference(['job_id', 'fraudulent'])

# Identify duplicates based on all columns except 'job_id' and 'fraudulent'
dup_mask = df.duplicated(subset=cols_to_check, keep=False)
df_duplicates = df[dup_mask].sort_values(by=list(cols_to_check))
print(f"Found {df_duplicates.shape[0]} duplicated rows (ignoring job_id and fraudulent).")

# Check if duplicated rows have consistent 'fraudulent' labels
inconsistent_labels = df.groupby(list(cols_to_check))['fraudulent'].nunique()
inconsistent_count = (inconsistent_labels > 1).sum()
print(f"Number of groups with inconsistent 'fraudulent' labels: {inconsistent_count}")


Found 527 duplicated rows (ignoring job_id and fraudulent).
Number of groups with inconsistent 'fraudulent' labels: 0


Presense of duplicated rows ignoring job_id & target label, duplicated rows all have the same target label

## Values for each row

### Identified columns to compare for uniqueness
- 'title'
- 'description'
- 'requirements'
- 'employment_type'
- 'location'

In [7]:
# Get value counts for each column
for column in df.columns:
    print(f"\n{column}:")
    print(df[column].value_counts().head())
    print(f"Null values: {df[column].isnull().sum()}")
    print(f"Unique values: {df[column].nunique()}")
    print("-" * 50)
    


job_id:
job_id
1        1
11919    1
11925    1
11924    1
11923    1
Name: count, dtype: int64
Null values: 0
Unique values: 17880
--------------------------------------------------

title:
title
English Teacher Abroad                                311
Customer Service Associate                            146
Graduates: English Teacher Abroad (Conversational)    144
English Teacher Abroad                                 95
Software Engineer                                      86
Name: count, dtype: int64
Null values: 0
Unique values: 11231
--------------------------------------------------

location:
location
GB, LND, London          718
US, NY, New York         658
US, CA, San Francisco    472
GR, I, Athens            464
US, ,                    339
Name: count, dtype: int64
Null values: 346
Unique values: 3105
--------------------------------------------------

department:
department
Sales          551
Engineering    487
Marketing      401
Operations     270
IT             225
N

In [8]:
# Further analyze interested columns
# show full value_counts (not truncated)
pd.set_option('display.max_rows', None)

for col in ['title', 'employment_type', 'location']:
    print(f"\n--- {col} ---")
    display(df[col].value_counts(dropna=False))

# restore default
pd.reset_option('display.max_rows')

# Show unique values for each column to identify "unknown" types
cols_to_check = ['required_experience', 'required_education', 'industry', 'function']

for col in cols_to_check:
    print(f"\nUnique values in '{col}':")
    print(df[col].value_counts(dropna=False))


--- title ---


title
English Teacher Abroad                                                                                                                            311
Customer Service Associate                                                                                                                        146
Graduates: English Teacher Abroad (Conversational)                                                                                                144
English Teacher Abroad                                                                                                                             95
Software Engineer                                                                                                                                  86
English Teacher Abroad (Conversational)                                                                                                            83
Customer Service Associate - Part Time                                                        


--- employment_type ---


employment_type
Full-time    11620
NaN           3471
Contract      1524
Part-time      797
Temporary      241
Other          227
Name: count, dtype: int64


--- location ---


location
GB, LND, London                                                                                                                                                                                  718
US, NY, New York                                                                                                                                                                                 658
US, CA, San Francisco                                                                                                                                                                            472
GR, I, Athens                                                                                                                                                                                    464
NaN                                                                                                                                                                                              346
US, , 


Unique values in 'required_experience':
required_experience
NaN                 7050
Mid-Senior level    3809
Entry level         2697
Associate           2297
Not Applicable      1116
Director             389
Internship           381
Executive            141
Name: count, dtype: int64

Unique values in 'required_education':
required_education
NaN                                  8105
Bachelor's Degree                    5145
High School or equivalent            2080
Unspecified                          1397
Master's Degree                       416
Associate Degree                      274
Certification                         170
Some College Coursework Completed     102
Professional                           74
Vocational                             49
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree                     6
Name: count, dtype: int64

Unique values in 'industry':
industry
N

,location_country,location_state,location_city
144,unknown,unknown,unknown
204,unknown,unknown,unknown
234,unknown,unknown,unknown
325,unknown,unknown,unknown
349,unknown,unknown,unknown
...,...,...,...
17792,unknown,unknown,unknown
17809,unknown,unknown,unknown
17821,unknown,unknown,unknown
17822,unknown,unknown,unknown


## Data Cleaning

NA values -> 'unknown'
Certain columns have other "NA" like values: eg. Not Applicable, NaN, Unspecified. They have all been replaced with 'unknown'

### Columns
location, employment_type, title, description and requirements are columns which are to be compared to find duplicates.


### Chunk Below
- Cleans columns ['required_experience', 'required_education', 'industry', 'function'] by replacing N/A values with str("unknown")
- Splits column ['location'] into 3 columns ['location_country', 'location_state', 'location_city']
- Handles employment type by:
    - Combining 'other' & 'unknown'
    - Combining 'temporary' & 'contract'
- Removes duplicates with enhanced restrictions on employment type & location

In [28]:
# Cleaning Employment Type for deduplication
df_cleaning = df.copy()

# Clean columns by replacing nulls and unspecified values with 'unknown'
cols_to_clean = ['required_experience', 'required_education', 'industry', 'function']
unspecified_values = ['Not Applicable','NaN','not applicable', 'Unspecified', 'Other','Others','none', 'na', 'n/a', '', ' ', None]

for col in cols_to_clean:
    df_cleaning[col] = df_cleaning[col].replace(unspecified_values, 'unknown')
    df_cleaning[col] = df_cleaning[col].fillna('unknown')

for col in df_cleaning.columns:
    df_cleaning[col] = df_cleaning[col].fillna('unknown')

# Adding columns for location cleaning: [country, state, city]
def clean_location(loc):
    if pd.isna(loc) or loc in unspecified_values:
        return ("unknown", "unknown", "unknown")
    parts = loc.split(',')
    parts = [part.strip() if part.strip() not in unspecified_values else "unknown" for part in parts]
    # Pad with "unknown" if we don't have all three parts
    while len(parts) < 3:
        parts.append("unknown")
    return (parts[0], parts[1], parts[2])
df_cleaning_loc = df_cleaning['location'].apply(clean_location)
df_cleaning['location_country'] = df_cleaning_loc.apply(lambda x: x[0])
df_cleaning['location_state'] = df_cleaning_loc.apply(lambda x: x[1])
df_cleaning['location_city'] = df_cleaning_loc.apply(lambda x: x[2])

def simplify_employment_type(x):
    if pd.isna(x):
        return 'unknown'
    x = x.strip().lower()
    if x in ['full-time', 'part-time']:
        return x  # keep these separate
    elif x in ['contract', 'temporary']:
        return 'non-permanent'
    elif x in ['other', 'unknown', '']:
        return 'unknown'
    else:
        return 'unknown'
df_cleaning['employment_type_clean'] = df_cleaning['employment_type'].apply(simplify_employment_type)

# Define a new function to compare locations accounting for unknowns
def compare_locations(row1, row2):
    # Compare countries first
    if row1['location_country'] is None or row2['location_country'] is None:
        return True
    if row1['location_country'] != row2['location_country']:
        return False
    # If countries match, compare states (unless either is unknown)
    if (row1['location_state'] != 'unknown' and 
        row2['location_state'] != 'unknown' and 
        row1['location_state'] != row2['location_state']):
        return False
    # If states match or either is unknown, compare cities (unless either is unknown)
    if (row1['location_city'] != 'unknown' and 
        row2['location_city'] != 'unknown' and 
        row1['location_city'] != row2['location_city']):
        return False
    return True

# Create a new comparison key function that uses the location comparison
def comparison_key(row):
    # Get location info with handling for unknown values
    location_info = (
        row['location_country'],
        row['location_state'] if row['location_state'] != 'unknown' else None,
        row['location_city'] if row['location_city'] != 'unknown' else None
    )
    
    # Get employment type, None if unknown
    emp = None if row['employment_type_clean'] == 'unknown' else row['employment_type_clean']
    
    # Return tuple with all comparison fields
    return (location_info, row['title'], row['description'], row['requirements'], emp)

df_cleaning['dedup_key'] = df_cleaning.apply(comparison_key, axis=1)
df_nodup = df_cleaning.drop_duplicates(subset=['dedup_key'])

df_nodup



,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,required_experience,required_education,industry,function,fraudulent,location_country,location_state,location_city,employment_type_clean,dedup_key
0,1,Marketing Intern,"US, NY, New York",Marketing,unknown,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,...,Internship,unknown,unknown,Marketing,0,US,NY,New York,unknown,"((US, NY, New York), Marketing Intern, Food52,..."
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,unknown,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,...,unknown,unknown,Marketing and Advertising,Customer Service,0,NZ,unknown,Auckland,full-time,"((NZ, None, Auckland), Customer Service - Clou..."
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",unknown,unknown,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,...,unknown,unknown,unknown,unknown,0,US,IA,Wever,unknown,"((US, IA, Wever), Commissioning Machinery Assi..."
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,unknown,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,...,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington,full-time,"((US, DC, Washington), Account Executive - Was..."
4,5,Bill Review Manager,"US, FL, Fort Worth",unknown,unknown,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,...,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth,full-time,"((US, FL, Fort Worth), Bill Review Manager, JO..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,unknown,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,...,Mid-Senior level,unknown,Computer Software,Sales,0,CA,ON,Toronto,full-time,"((CA, ON, Toronto), Account Director - Distrib..."
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,unknown,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,...,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0,US,PA,Philadelphia,full-time,"((US, PA, Philadelphia), Payroll Accountant, T..."
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",unknown,unknown,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,unknown,0,...,unknown,unknown,unknown,unknown,0,US,TX,Houston,full-time,"((US, TX, Houston), Project Cost Control Staff..."
17878,17879,Graphic Designer,"NG, LA, Lagos",unknown,unknown,unknown,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,...,unknown,Professional,Graphic Design,Design,0,NG,LA,Lagos,non-permanent,"((NG, LA, Lagos), Graphic Designer, Nemsia Stu..."


In [6]:
# Binary check
binary_cols = [col for col in df.columns if df[col].nunique() == 2]
df_binary = df[binary_cols]
df_binary = df_binary.apply(lambda x: x.map({'Yes': 1, 'No': 0, True: 1, False: 0}) 
                            if x.dtypes == 'object' else x)
corr_matrix = df_binary.corr()
print(corr_matrix)

                  telecommuting  has_company_logo  has_questions  fraudulent
telecommuting          1.000000         -0.019836       0.020345    0.034523
has_company_logo      -0.019836          1.000000       0.233932   -0.261971
has_questions          0.020345          0.233932       1.000000   -0.091627
fraudulent             0.034523         -0.261971      -0.091627    1.000000


Logo & Fraudulent inverse relationship

In [7]:
# perform TD-IDF with job title and compare with fraudulent column
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['title'].fillna(''))
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df['fraudulent'] = df['fraudulent'].values
correlations = tfidf_df.corr()['fraudulent'].sort_values(ascending=False)

In [8]:
# most common words in title associated with fraudulent job postings
print(correlations)

fraudulent    1.000000
earn          0.183713
payroll       0.183350
entry         0.182795
daily         0.181072
                ...   
designer     -0.036949
abroad       -0.044762
teacher      -0.047209
english      -0.047576
developer    -0.058592
Name: fraudulent, Length: 4567, dtype: float64
